In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark import SparkContext
import os

In [3]:
os.environ["SPARK_VERSION"]='3.3'

In [11]:
from pydeequ.analyzers import AnalysisRunner

from pydeequ.analyzers import AnalyzerContext
from pydeequ import deequ_maven_coord
from pydeequ import f2j_maven_coord

In [5]:
# create spark session and spark context
spark = (SparkSession
    .builder
    .config("spark.jars.packages", deequ_maven_coord)
    .config("spark.jars.excludes", f2j_maven_coord)
    .getOrCreate())
sc = SparkContext.getOrCreate()


23/08/28 21:26:36 WARN Utils: Your hostname, kvothe resolves to a loopback address: 127.0.1.1; using 192.168.100.21 instead (on interface wlp2s0)
23/08/28 21:26:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/senoj/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/senoj/.ivy2/cache
The jars for the packages stored in: /home/senoj/.ivy2/jars
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c3c70c8e-e1cd-43a3-8d73-66c5389f62a4;1.0
	confs: [default]
	found com.amazon.deequ#deequ;2.0.3-spark-3.3 in central
	found org.scala-lang#scala-reflect;2.12.10 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransforms;2.4.0 in central
	found junit#junit;4.8.2 in central
	found org.apache.commons#commons-math3;3.2 in central
	found org.spire-math#spire_2.12;0.13.0 in central
	found org.spire-math#spire-macros_2.12;0.13.0 in central
	found org.typelevel#machinist_2.12;0.6.1 in central
	found com.chuusai#shapeless_2.12;2.3.2 in central
	found org.typelevel#macro-compat_2.12;1.1.1 in

In [8]:
df

DataFrame[Segmento: string, CNPJ: string, Nome: string]

In [6]:
df: DataFrame = spark.read.csv("./app/data/Bancos", header=True, sep="\t")

In [28]:
from pydeequ.analyzers import Size
from pydeequ.analyzers import Completeness
from pydeequ.analyzers import CountDistinct
from pydeequ.analyzers import Distinctness

analysis_result = AnalysisRunner(spark).onData(df)\
                                            .addAnalyzer(Size())\
                                            .addAnalyzer(Completeness("Segmento"))\
                                            .addAnalyzer(Completeness("CNPJ"))\
                                            .addAnalyzer(CountDistinct("CNPJ"))\
                                            .addAnalyzer(Distinctness("CNPJ"))\
                                            .run()
analysis_result_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysis_result)
analysis_result_df.show()

+-------+--------+-------------+------------------+
| entity|instance|         name|             value|
+-------+--------+-------------+------------------+
| Column|    CNPJ|CountDistinct|            1459.0|
|Dataset|       *|         Size|            1474.0|
| Column|Segmento| Completeness|               1.0|
| Column|    CNPJ| Distinctness|0.9898236092265943|
| Column|    CNPJ| Completeness|               1.0|
+-------+--------+-------------+------------------+



In [26]:
from pydeequ.checks import Check
from pydeequ.checks import CheckLevel
from pydeequ.verification import VerificationSuite
from pydeequ.verification import VerificationResult
check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.isContainedIn("Segmento", ["S1", "S2", "S3", "S4", "S5"])\
             .hasCompleteness("CNPJ", lambda x: x >= 0.98))\
    .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

+------------+-----------+------------+--------------------+-----------------+--------------------+
|       check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+------------+-----------+------------+--------------------+-----------------+--------------------+
|Review Check|    Warning|     Warning|ComplianceConstra...|          Success|                    |
|Review Check|    Warning|     Warning|CompletenessConst...|          Failure|Value: 1.0 does n...|
+------------+-----------+------------+--------------------+-----------------+--------------------+

